In [1]:
import time

from pathlib import Path

import plotly.express as px
import plotly.graph_objects as go
import polars as pl

from plotly.subplots import make_subplots
from result import Result

from raft.io import read_file
from raft.utils.log import logger

In [2]:
from raft.services.sirius_messages import (
    ImageCaptureMessage,
    SeabirdCTDMessage,
    AanderaaCTDMessage,
    EcopuckMessage,

    ParosciPressureMessage,
    TeledyneDVLMessage,
    LQModemMessage,
    EvologicsModemMessage,

    MicronSonarMessage,
    OASonarMessage,

    BatteryMessage,
    ThrusterMessage,
)

from raft.services.sirius_messages import AuvMessageHeader, parse_message_header
from raft.services.sirius_messages import MESSAGE_TYPE_PARSERS

### Parse all messages

In [3]:
paths: dict = dict()

paths["datadir"] = Path(
    "/media/martin/pcie_01/acfr_revisits_processed/acfr_merged_message_files"
)
paths["messages"] = paths["datadir"] / Path("qd61g27j_20170523_040815_messages.txt")

lines: list[str] = read_file(paths["messages"]).unwrap()

# Mapping from topic to message type
TOPIC_TO_TYPE: dict[str, type] = {
    "VIS": ImageCaptureMessage,
    "SEABIRD": SeabirdCTDMessage,
    "AANDERAA_4317": AanderaaCTDMessage,
    "ECOPUCK": EcopuckMessage,

    "PAROSCI": ParosciPressureMessage,
    "RDI": TeledyneDVLMessage,
    "LQMODEM": LQModemMessage,
    "EVOLOGICS_FIX": EvologicsModemMessage,

    "MICRON": MicronSonarMessage,
    "MICRON_RETURNS": MicronSonarMessage,
    "OAS": OASonarMessage,

    "BATT": BatteryMessage,
    "BATT0": BatteryMessage,
    "BATT1": BatteryMessage,
    "BATT2": BatteryMessage,
    
    "THR_PORT": ThrusterMessage,
    "THR_STBD": ThrusterMessage,
    "THR_VERT": ThrusterMessage,
}

def parse_messages(lines: list[str]) -> dict:
    """TODO"""
    parsed_messages: dict[type, list] = dict()
    for line in lines:
        header: AuvMessageHeader = parse_message_header(line).unwrap()
    
        if not header.topic in TOPIC_TO_TYPE:
            continue
        
        message_type: type = TOPIC_TO_TYPE[header.topic]    
        message: type = DEFAULT_MESSAGE_PARSERS[message_type](line).unwrap()
    
        if message_type not in parsed_messages:
            parsed_messages[message_type] = list()
        
        parsed_messages[message_type].append(message)
        
    return parsed_messages

parsed_messages: dict = parse_messages(lines)

for message_type, messages in parsed_messages.items():
    logger.info(f"{message_type}: {len(messages)}")

for message_type, messages in parsed_messages.items():
    logger.info("")
    logger.info(f"{messages[0]}")

2024-06-07 22:44:32.031 | INFO     | __main__:<module>:58 - <class 'raft.services.sirius_messages.data_types.MicronSonarMessage'>: 168123
2024-06-07 22:44:32.032 | INFO     | __main__:<module>:58 - <class 'raft.services.sirius_messages.data_types.ParosciPressureMessage'>: 6580
2024-06-07 22:44:32.032 | INFO     | __main__:<module>:58 - <class 'raft.services.sirius_messages.data_types.EcopuckMessage'>: 7579
2024-06-07 22:44:32.032 | INFO     | __main__:<module>:58 - <class 'raft.services.sirius_messages.data_types.EvologicsModemMessage'>: 41
2024-06-07 22:44:32.033 | INFO     | __main__:<module>:58 - <class 'raft.services.sirius_messages.data_types.TeledyneDVLMessage'>: 30408
2024-06-07 22:44:32.033 | INFO     | __main__:<module>:58 - <class 'raft.services.sirius_messages.data_types.BatteryMessage'>: 4266
2024-06-07 22:44:32.033 | INFO     | __main__:<module>:58 - <class 'raft.services.sirius_messages.data_types.ThrusterMessage'>: 227348
2024-06-07 22:44:32.033 | INFO     | __main__:<mo